In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [4]:
# start the Ollama server in background
!nohup ollama serve > output.log 2>&1 &

In [5]:
!ollama pull phi4-mini:3.8b

In [6]:
!ollama list

NAME              ID              SIZE      MODIFIED               
phi4-mini:3.8b    78fad5d182a7    2.5 GB    Less than a second ago    


In [7]:
!pip install pydantic ollama

In [8]:
from pydantic import BaseModel, Field
from typing import List
import ollama

# Define FAQ entry
class FAQEntry(BaseModel):
  question: str = Field(..., description="The FAQ question")
  answer: str = Field(..., description="The FAQ answer")

# Define the full FAQ dataset model
class FAQData(BaseModel):
  faqs: List[FAQEntry]

  def format_for_prompt(self) -> str:
    return "\n".join([f"Q: {faq.question}\nA: {faq.answer}" for faq in self.faqs])

#Sample FAQ data
faq_data = FAQData(
    faqs=[
        FAQEntry(question="What is an AI Agent?", answer="An AI agent is a system that can autonomously perform tasks based on inputs and learned by knowledge"),
        FAQEntry(question="How can AI agents improve efficiency?", answer="AI agents can automate repetitive tasks, streamline decision-making, and enhance customer support with accurate responses."),
        FAQEntry(question="What industries benefit most from AI agents?", answer="Industries like healthcare, e-commerce, finance, and customer support heavily benefit from AI agents."),
        FAQEntry(question="Are AI agents customizable?", answer="Yes, AI agents can be trained with specific data and prompts to suit various use cases.")
    ]
)

# Chatbot function
def chatbot():
  print("Welcome to the AI FAQ Chatbot (Phi-4 via Ollama + Pydantic)! Ask about agents and their benefits. Type 'exit' to quit.")
  faq_context = faq_data.format_for_prompt()

  while True:
    user_input = input("You: ").strip()
    if user_input.lower() == 'exit':
      print("Goodbye!")
      break

    #Build a prompt
    prompt = (f"""You are an AI chatbot. Answer shortly and precisely to the point. Answer this question based on the provided FAQ data:\n
    {faq_context}\n\n
    Question: {user_input}\n
    Answer:""")
    response = ollama.chat(model="phi4-mini:3.8b", messages=[{"role": "user", "content": prompt}])
    print("Bot: ", response["message"]["content"])

#Call the function
display(chatbot())

Welcome to the AI FAQ Chatbot (Phi-4 via Ollama + Pydantic)! Ask about agents and their benefits. Type 'exit' to quit.
You: hi
Bot:  Hi! How I assist you today?
You: what is LLM 
Bot:  LLM stands for Large Language Model. It's a type of artificial intelligence designed to understand and generate human-like text based on the context provided. These models are commonly used in natural language processing tasks such as translation, summarization, question answering, etc. They have been trained with vast amounts of textual data from various sources like books, websites, articles documents which enable them to comprehend complex sentence structures, idioms expressions and even generate coherent responses for given prompts or queries. Examples include GPT-3 developed by Microsoft and Phi developed by Microsoft also known as Phi language model are examples of LLMs that have been widely recognized due their advanced capabilities in generating human-like text and providing helpful outputs based

None